In [1]:
import json

In [2]:
# Generated by mass2lda
lda_dict = json.load(open('METABOLOMICS-SNETS-e9eaf6ec-download_clustered_spectra-main.lda.json'))
corpus = json.load(open('METABOLOMICS-SNETS-e9eaf6ec-download_clustered_spectra-main.corpus.json'))

In [3]:
K = 300
n_its = 1000

In [4]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="mass2lda")

In [7]:
sc

<SparkContext master=local[*] appName=mass2lda>

In [8]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

In [9]:
m = []
for doc, words in corpus.items():
    scores = {lda_dict['word_index'][word]: score for word, score in words.items()}
    m.append([lda_dict['doc_index'][doc], Vectors.sparse(len(lda_dict['word_index']), scores)])

In [10]:
tf = sc.parallelize(m)

In [18]:
m

[[0, SparseVector(546, {0: 787.0, 1: 1000.0, 2: 787.0, 3: 1000.0})],
 [1, SparseVector(546, {4: 1000.0, 5: 1000.0})],
 [2, SparseVector(546, {6: 1000.0, 7: 1000.0})],
 [3,
  SparseVector(546, {8: 764.0, 9: 764.0, 10: 451.0, 11: 1000.0, 12: 1000.0, 13: 451.0})],
 [4,
  SparseVector(546, {14: 821.0, 15: 844.0, 16: 1000.0, 17: 821.0, 18: 844.0})],
 [5,
  SparseVector(546, {16: 1000.0, 19: 201.0, 20: 170.0, 21: 114.0, 22: 114.0, 23: 154.0, 24: 170.0, 25: 201.0, 26: 154.0})],
 [6,
  SparseVector(546, {16: 664.0, 18: 292.0, 27: 552.0, 28: 174.0, 29: 292.0, 30: 552.0, 31: 294.0, 32: 1000.0, 33: 294.0, 34: 174.0, 35: 1000.0, 36: 664.0})],
 [7, SparseVector(546, {37: 1000.0, 38: 1000.0})],
 [8, SparseVector(546, {39: 767.0, 40: 1000.0, 41: 767.0, 42: 1000.0})],
 [9, SparseVector(546, {43: 1000.0})],
 [10, SparseVector(546, {38: 1000.0, 44: 1000.0})],
 [11, SparseVector(546, {45: 1000.0, 46: 1000.0})],
 [12, SparseVector(546, {47: 802.0, 48: 1000.0, 49: 802.0, 50: 1000.0})],
 [13,
  SparseVector

In [16]:
ldaModel = LDA.train(tf, k=3, maxIterations=40)

In [12]:
ldaModel.topicsMatrix()

array([[3.93212608e-04, 1.69499919e+03, 4.19118401e-04],
       [2.90521696e+02, 1.01147006e+03, 8.23931589e-03],
       [4.61538172e-04, 7.86999058e+02, 4.80735934e-04],
       ...,
       [7.71519732e-04, 9.99998422e+02, 8.06447318e-04],
       [7.71206964e-04, 1.98998423e+02, 8.06120296e-04],
       [1.80515072e-03, 1.59709599e-03, 9.99996598e+02]])

In [17]:
# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())
      + " words):")
topics = ldaModel.topicsMatrix()
for topic in range(3):
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        print(" " + str(topics[word][topic]))


Learned topics (as distributions over vocab of 546 words):
Topic 0:
 1614.8214720037572
 1008.8970146856001
 786.9990206551607
 1040.774440182875
 2083.8425650574864
 999.9985526869311
 0.0009387555105591423
 0.0009387081371413412
 607.4949194526439
 179.1686879221138
 0.0019085055028394112
 0.00079172724592159
 915.5609891538661
 0.0006022579407464481
 173.91164459854966
 0.0005046041911861616
 1247.2961471287465
 0.0005046025080114087
 0.5122827582458487
 0.001722313724895318
 0.0009981390452202925
 0.0017809496526096727
 0.04605266236421972
 0.0023402437150268824
 0.001722154565601083
 583.7701750030892
 0.0017220473632374464
 401.0720509715976
 173.98410869174268
 291.98410444701204
 551.9841014977048
 293.9841044044228
 999.9841000134402
 245.5263429844747
 900.7343877146171
 999.9841000134402
 663.8741506537775
 0.0009386990969247045
 0.0007934730456129219
 0.0005552474518990255
 0.020502091728369823
 0.0006493434618288498
 0.0005552644113395334
 17999.988242800548
 0.00093869909

In [30]:
data = sc.textFile("../spark-2.3.1-bin-hadoop2.7/data/mllib/sample_lda_data.txt")

In [31]:
parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))

In [32]:
txtCorpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]])

In [34]:
txtCorpus.collect()

[[0, DenseVector([1.0, 2.0, 6.0, 0.0, 2.0, 3.0, 1.0, 1.0, 0.0, 0.0, 3.0])],
 [1, DenseVector([1.0, 3.0, 0.0, 1.0, 3.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0])],
 [2, DenseVector([1.0, 4.0, 1.0, 0.0, 0.0, 4.0, 9.0, 0.0, 1.0, 2.0, 0.0])],
 [3, DenseVector([2.0, 1.0, 0.0, 3.0, 0.0, 0.0, 5.0, 0.0, 2.0, 3.0, 9.0])],
 [4, DenseVector([3.0, 1.0, 1.0, 9.0, 3.0, 0.0, 2.0, 0.0, 0.0, 1.0, 3.0])],
 [5, DenseVector([4.0, 2.0, 0.0, 3.0, 4.0, 5.0, 1.0, 1.0, 1.0, 4.0, 0.0])],
 [6, DenseVector([2.0, 1.0, 0.0, 3.0, 0.0, 0.0, 5.0, 0.0, 2.0, 2.0, 9.0])],
 [7, DenseVector([1.0, 1.0, 1.0, 9.0, 2.0, 1.0, 2.0, 0.0, 0.0, 1.0, 3.0])],
 [8, DenseVector([4.0, 4.0, 0.0, 3.0, 4.0, 2.0, 1.0, 3.0, 0.0, 0.0, 0.0])],
 [9, DenseVector([2.0, 8.0, 2.0, 0.0, 3.0, 0.0, 2.0, 0.0, 2.0, 7.0, 2.0])],
 [10, DenseVector([1.0, 1.0, 1.0, 9.0, 0.0, 2.0, 2.0, 0.0, 0.0, 3.0, 3.0])],
 [11, DenseVector([4.0, 1.0, 0.0, 0.0, 4.0, 5.0, 1.0, 3.0, 0.0, 1.0, 0.0])]]